In [485]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
from collections import Counter
from sklearn.metrics import accuracy_score

In [486]:
weather = pd.read_csv("../Tan - Random Forest/hanoiweather.csv")

In [487]:
weather

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,NaN,NaN,NaN,1990-01-01T06:33:41,1990-01-01T17:26:18,0.14,Overcast,Cloudy skies throughout the day.,cloudy,48820099999
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,NaN,NaN,NaN,1990-01-02T06:34:00,1990-01-02T17:26:56,0.18,Overcast,Cloudy skies throughout the day.,cloudy,48820099999
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,NaN,NaN,NaN,1990-01-03T06:34:18,1990-01-03T17:27:34,0.22,Overcast,Cloudy skies throughout the day.,cloudy,48820099999
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,NaN,NaN,NaN,1990-01-04T06:34:35,1990-01-04T17:28:12,0.25,Overcast,Cloudy skies throughout the day.,cloudy,48820099999
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,NaN,NaN,NaN,1990-01-05T06:34:51,1990-01-05T17:28:51,0.29,Overcast,Cloudy skies throughout the day.,cloudy,48820099999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12534,Hà Nội,2024-04-26,36.0,27.0,30.9,47.9,31.0,39.6,26.9,80.8,...,24.9,10.0,60.0,2024-04-26T05:29:23,2024-04-26T18:19:42,0.58,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,VVNB
12535,Hà Nội,2024-04-27,40.0,27.0,32.2,44.7,31.2,38.6,24.7,69.9,...,28.6,10.0,30.0,2024-04-27T05:28:42,2024-04-27T18:20:04,0.61,Clear,Clear conditions throughout the day.,clear-day,"48820099999,48823099999,48825099999,4883109999..."
12536,Hà Nội,2024-04-28,35.0,27.0,30.4,50.8,30.5,39.4,27.0,82.7,...,28.1,10.0,60.0,2024-04-28T05:28:02,2024-04-28T18:20:27,0.65,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48820099999,48823099999,48825099999,4883109999..."
12537,Hà Nội,2024-04-29,34.0,28.0,30.2,49.3,34.4,39.4,27.4,85.3,...,25.3,10.0,75.0,2024-04-29T05:27:22,2024-04-29T18:20:49,0.68,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,VVNB


In [488]:
# Tạo cột mới có giá trị là nhiệt độ cao nhất của ngày hôm sau 
weather['tempmax_next'] = weather['tempmax'].shift(-1)

In [489]:
weather.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,tempmax_next
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,NaN,NaN,1990-01-01T06:33:41,1990-01-01T17:26:18,0.14,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,17.4
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,NaN,NaN,1990-01-02T06:34:00,1990-01-02T17:26:56,0.18,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,18.2
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,NaN,NaN,1990-01-03T06:34:18,1990-01-03T17:27:34,0.22,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,19.6
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,NaN,NaN,1990-01-04T06:34:35,1990-01-04T17:28:12,0.25,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,18.1
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,NaN,NaN,1990-01-05T06:34:51,1990-01-05T17:28:51,0.29,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,17.7


In [490]:
# Xử lý dữ liệu khuyết do hàng cuối sẽ là NaN => bỏ dòng cuối
weather = weather[:-1]

In [491]:
weather.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,tempmax_next
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,NaN,NaN,1990-01-01T06:33:41,1990-01-01T17:26:18,0.14,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,17.4
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,NaN,NaN,1990-01-02T06:34:00,1990-01-02T17:26:56,0.18,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,18.2
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,NaN,NaN,1990-01-03T06:34:18,1990-01-03T17:27:34,0.22,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,19.6
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,NaN,NaN,1990-01-04T06:34:35,1990-01-04T17:28:12,0.25,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,18.1
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,NaN,NaN,1990-01-05T06:34:51,1990-01-05T17:28:51,0.29,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,17.7


In [492]:
# Mã hoá Conditions vì là dữ liệu phân loại 
conditions_encoded = pd.get_dummies(weather['conditions'], prefix='cond')
weather= pd.concat([weather, conditions_encoded], axis=1)

In [493]:
# Lựa chọn các loại đặc trưng độ ẩm, lớp phủ mây, tốc độ gió,  áp suất mực nước biển
features = ['tempmax', 'tempmin', 'humidity', 'cloudcover', 'windspeed', 'sealevelpressure']
X = weather[features]
y = weather['tempmax_next']

In [494]:
#Thêm condition vào features 
condition_features = [col for col in weather.columns if 'cond_' in col]
features.extend(condition_features)

In [495]:
features

['tempmax',
 'tempmin',
 'humidity',
 'cloudcover',
 'windspeed',
 'sealevelpressure',
 'cond_Clear',
 'cond_Overcast',
 'cond_Partially cloudy',
 'cond_Rain',
 'cond_Rain, Overcast',
 'cond_Rain, Partially cloudy']

In [496]:
weather.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,description,icon,stations,tempmax_next,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,Cloudy skies throughout the day.,cloudy,48820099999,17.4,False,True,False,False,False,False
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,Cloudy skies throughout the day.,cloudy,48820099999,18.2,False,True,False,False,False,False
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,Cloudy skies throughout the day.,cloudy,48820099999,19.6,False,True,False,False,False,False
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,Cloudy skies throughout the day.,cloudy,48820099999,18.1,False,True,False,False,False,False
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,Cloudy skies throughout the day.,cloudy,48820099999,17.7,False,True,False,False,False,False


In [497]:
def entropy(s):
    values, counts = np.unique(s, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

In [498]:
tempmax_entropy = entropy(weather['tempmax'])
print("Entropy of tempmax:", tempmax_entropy)

Entropy of tempmax: 7.015482289848042


In [499]:
X = weather[features]
y = weather['tempmax_next']

In [500]:
print("Số lượng giá trị khuyết trong X:")
print(X.isnull().sum())
print("\nSố lượng giá trị khuyết trong y:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong X:
tempmax                        57
tempmin                        57
humidity                       57
cloudcover                     57
windspeed                       0
sealevelpressure               99
cond_Clear                      0
cond_Overcast                   0
cond_Partially cloudy           0
cond_Rain                       0
cond_Rain, Overcast             0
cond_Rain, Partially cloudy     0
dtype: int64

Số lượng giá trị khuyết trong y:
57


In [501]:
weather_clean = weather.dropna(subset=features + ['tempmax_next'])
X = weather_clean[features]
y = weather_clean['tempmax_next']

In [502]:
print("Số lượng giá trị khuyết trong X:")
print(X.isnull().sum())
print("\nSố lượng giá trị khuyết trong y:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong X:
tempmax                        0
tempmin                        0
humidity                       0
cloudcover                     0
windspeed                      0
sealevelpressure               0
cond_Clear                     0
cond_Overcast                  0
cond_Partially cloudy          0
cond_Rain                      0
cond_Rain, Overcast            0
cond_Rain, Partially cloudy    0
dtype: int64

Số lượng giá trị khuyết trong y:
0


In [503]:
# Kiểm tra kiểu dữ liệu của các cột trong `X`
print("\nKiểu dữ liệu của các cột trong X:")
print(X.dtypes)



Kiểu dữ liệu của các cột trong X:
tempmax                        float64
tempmin                        float64
humidity                       float64
cloudcover                     float64
windspeed                      float64
sealevelpressure               float64
cond_Clear                        bool
cond_Overcast                     bool
cond_Partially cloudy             bool
cond_Rain                         bool
cond_Rain, Overcast               bool
cond_Rain, Partially cloudy       bool
dtype: object


In [504]:
# Thống kê mô tả cho `X`
print("\nThống kê mô tả cho X:")
print(X.describe())

# Thống kê mô tả cho `y`
print("\nThống kê mô tả cho y:")
print(y.describe())


Thống kê mô tả cho X:
            tempmax       tempmin      humidity    cloudcover     windspeed  \
count  12436.000000  12436.000000  12436.000000  12436.000000  12436.000000   
mean      27.765704     21.403916     80.848191     70.037544     17.380018   
std        5.812475      4.984567     10.364983     25.272475      6.242558   
min        8.000000      3.800000     31.100000      0.000000      0.000000   
25%       24.000000     18.000000     75.000000     56.300000     13.200000   
50%       29.000000     23.000000     82.100000     77.200000     16.600000   
75%       32.200000     25.100000     88.300000     90.100000     21.400000   
max       43.100000     34.600000    100.000000    100.000000     79.200000   

       sealevelpressure  
count      12436.000000  
mean        1010.874405  
std            7.019944  
min          991.100000  
25%         1005.200000  
50%         1010.600000  
75%         1016.300000  
max         1037.200000  

Thống kê mô tả cho y:
count   

In [505]:
# Phân chia tập dữ liệu
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Kích thước tập huấn luyện:", X_train.shape, y_train.shape)
print("Kích thước tập kiểm thử:", X_test.shape, y_test.shape)

Kích thước tập huấn luyện: (9948, 12) (9948,)
Kích thước tập kiểm thử: (2488, 12) (2488,)


In [506]:
print("Số lượng giá trị khuyết trong y sau khi loại bỏ:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong y sau khi loại bỏ:
0


In [507]:
from sklearn.ensemble import RandomForestRegressor

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)  # Sử dụng 100 cây

# Huấn luyện mô hình trên tập huấn luyện
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [508]:
from sklearn.metrics import mean_squared_error, r2_score

# Dự đoán nhiệt độ trên tập kiểm thử
y_pred = model.predict(X_test)

# Đánh giá mô hình bằng Mean Squared Error và R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 5.196573974678458
R-squared: 0.8459301255691873


In [511]:
weather

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,description,icon,stations,tempmax_next,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,Cloudy skies throughout the day.,cloudy,48820099999,17.4,False,True,False,False,False,False
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,Cloudy skies throughout the day.,cloudy,48820099999,18.2,False,True,False,False,False,False
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,Cloudy skies throughout the day.,cloudy,48820099999,19.6,False,True,False,False,False,False
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,Cloudy skies throughout the day.,cloudy,48820099999,18.1,False,True,False,False,False,False
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,Cloudy skies throughout the day.,cloudy,48820099999,17.7,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12533,Hà Nội,2024-04-25,34.0,25.0,28.9,45.6,25.0,34.5,25.9,84.7,...,Partly cloudy throughout the day with early mo...,rain,"48820099999,48823099999,48825099999,4883109999...",36.0,False,False,False,False,False,True
12534,Hà Nội,2024-04-26,36.0,27.0,30.9,47.9,31.0,39.6,26.9,80.8,...,Clearing in the afternoon.,partly-cloudy-day,VVNB,40.0,False,False,True,False,False,False
12535,Hà Nội,2024-04-27,40.0,27.0,32.2,44.7,31.2,38.6,24.7,69.9,...,Clear conditions throughout the day.,clear-day,"48820099999,48823099999,48825099999,4883109999...",35.0,True,False,False,False,False,False
12536,Hà Nội,2024-04-28,35.0,27.0,30.4,50.8,30.5,39.4,27.0,82.7,...,Partly cloudy throughout the day.,partly-cloudy-day,"48820099999,48823099999,48825099999,4883109999...",34.0,False,False,True,False,False,False


In [512]:
#Dự đoán nhiệt độ cao nhất ngày 1/5 từ nhiệt độ 29/4 do 30/4 đã bị mất
data = {
    'tempmax': [34],
    'tempmin': [28],
    'humidity': [85.3],
    'cloudcover': [72.9],
    'windspeed': [20.5],
    'sealevelpressure': [1002],
    'cond_Clear': [0],
    'cond_Overcast': [0],
    'cond_Partially cloudy': [1],
    'cond_Rain': [0],
    'cond_Rain, Overcast': [0],
    'cond_Rain, Partially cloudy': [0]
}

new_data = pd.DataFrame(data)

In [513]:
new_data

,tempmax,tempmin,humidity,cloudcover,windspeed,sealevelpressure,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,34,28,85.3,72.9,20.5,1002,0,0,1,0,0,0


In [514]:
predicted_tempmax = model.predict(new_data)
print(f"Dự đoán nhiệt độ cao nhất cho ngày 1 tháng 5 tại Hà Nội là: {predicted_tempmax[0]}°C")

Dự đoán nhiệt độ cao nhất cho ngày 1 tháng 5 tại Hà Nội là: 33.734999999999964°C
